In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import nltk
import warnings
warnings.filterwarnings('ignore')
from sklearn import metrics
from nltk.corpus import stopwords

In [ ]:
df_orig = pd.read_csv("/content/drive/MyDrive/SEM_8_Lab/NLP Package/UpdatedResumeDataSet1.csv")
df = df_orig.copy(deep = True)

In [ ]:
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


### Data cleaning

In [ ]:
df.columns    #list of columns

Index(['Category', 'Resume'], dtype='object')

In [ ]:
df.columns.isnull()     #no null values

array([False, False])

In [ ]:
df.shape  #checking number of rows,column

(962, 2)

In [ ]:
print ("Displaying the distinct categories of resume -") # display unique domains
print (df['Category'].unique())

Displaying the distinct categories of resume -
['Data Science' 'HR' 'Advocate' 'Arts' 'Web Designing'
 'Mechanical Engineer' 'Sales' 'Health and fitness' 'Civil Engineer'
 'Java Developer' 'Business Analyst' 'SAP Developer' 'Automation Testing'
 'Electrical Engineering' 'Operations Manager' 'Python Developer'
 'DevOps Engineer' 'Network Security Engineer' 'PMO' 'Database' 'Hadoop'
 'ETL Developer' 'DotNet Developer' 'Blockchain' 'Testing']


In [ ]:
# display unique domains and count of resumes on each domain
print ("Displaying the distinct categories of resume and the number of records belonging to each category -")
print (df['Category'].value_counts())

Displaying the distinct categories of resume and the number of records belonging to each category -
Java Developer               84
Testing                      70
DevOps Engineer              55
Python Developer             48
Web Designing                45
HR                           44
Hadoop                       42
Blockchain                   40
ETL Developer                40
Operations Manager           40
Data Science                 40
Sales                        40
Mechanical Engineer          40
Arts                         36
Database                     33
Electrical Engineering       30
Health and fitness           30
PMO                          30
Business Analyst             28
DotNet Developer             28
Automation Testing           26
Network Security Engineer    25
SAP Developer                24
Civil Engineer               24
Advocate                     20
Name: Category, dtype: int64


In [ ]:
#count of words in the dataset before data cleaning
with open('/content/drive/MyDrive/SEM_8_Lab/NLP Package/UpdatedResumeDataSet1.csv', 'r') as file:
    file_contents = file.read()

    print('Total words:   ', len(file_contents.split()))

Total words:    434127


In [ ]:
df['newer_res'] = '' # creating new columne to keep the cleaned

In [ ]:
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', ' ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText)
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*') #removing stopwords
    resumeText = pattern.sub('', resumeText)
    resumeText = re.sub(r"[A-Za-z]",
                    lambda x :  x.group(0).lower()
                                if x.group(0).isupper()       #uppercase to lowercase
                                else x.group(0).lower(),
                    resumeText)
    stop_words_lst = ['description', 'mumbai', 'chennai', 'location', 'karate', 'january', 'august', 'nagpur', 'months',
                 'solapur', 'maharashtra', 'march', 'district', 'using']
    for w in stop_words_lst:                                    #removing custom stopwords
        pattern = r'\b'+w+r'\b'
        resumeText = re.sub(pattern, '', resumeText)
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace

    return resumeText

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df['newer_res'] = df.Resume.apply(lambda x: cleanResume(x))
print (df['newer_res'][29])

expertise data quantitative analysis decision analytics predictive modeling data driven personalization kpi dashboards big data queries interpretation data mining visualization tools machine learning algorithms business intelligence bi research reports forecasts education details pgp data science aegis school data science business b e electronics communication electronics communication indore madhya pradesh ies ips academy data scientist data scientist pr canada skill details algorithms exprience 6 bi exprience 6 business intelligence exprience 6 machine learning exprience 24 visualization exprience 24 spark exprience 24 python exprience 36 tableau exprience 36 data analysis exprience 24 monthscompany details company aegis school data science business mostly working industry project providing solution along teaching appointments teach undergraduate graduate level courses spark machine learning adjunct faculty member aegis school data science 2017 present company aegis school data busin

In [ ]:
df.head()

,Category,Resume,category_id,newer_res
0,Data Science,skills programming languages python pandas num...,0,skills programming languages python pandas num...
1,Data Science,education details may 2013 may 2017 b e uit rg...,0,education details may 2013 may 2017 b e uit rg...
2,Data Science,areas interest deep learning control system de...,0,areas interest deep learning control system de...
3,Data Science,skills r python sap hana tableau sap hana sql ...,0,skills r python sap hana tableau sap hana sql ...
4,Data Science,education details mca ymcaust faridabad haryan...,0,education details mca ymcaust faridabad haryan...


In [ ]:
a = df.Resume.apply(len)
a

0      3951
1       974
2      1450
3      5684
4       327
       ... 
957    1686
958    1776
959    2500
960     608
961    1557
Name: Resume, Length: 962, dtype: int64

In [ ]:
df['resume_len'] = df.Resume.apply(len)
# df['resume_len'] = df.Resume.apply(len)
# sns.distplot(df.resume_len);
# a

In [ ]:
df.to_csv("clean_data3.csv")
print("Cleaned dataset written to cleaned_data.csv")

Cleaned dataset written to cleaned_data.csv


### checking the similarity between resumes

In [ ]:
import pandas as pd
df = pd.read_csv('clean_data1.csv')
df = df.drop(['Resume'],axis=1)
df.rename(columns={'newer_res':'Resume'},inplace=True)



#some cleaning
from io import StringIO
col = ['Category', 'Resume']
df = df[col]
df = df[pd.notnull(df['Resume'])]
df.columns = ['Category', 'Resume']
df['category_id'] = df['Category'].factorize()[0]
category_id_df = df[['Category', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Category']].values)

df.head()

,Category,Resume,category_id
0,Data Science,skills programming languages python pandas num...,0
1,Data Science,education details may 2013 may 2017 b e uit rg...,0
2,Data Science,areas interest deep learning control system de...,0
3,Data Science,skills r python sap hana tableau sap hana sql ...,0
4,Data Science,education details mca ymcaust faridabad haryan...,0


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(df.Resume)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names_out()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

,000,01,017,02,03,04,04th,05,050education,06th,...,zaggle,zambia,zd,zenoss,zensar,zero,zhypility,zone,zookeeper,zz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df

,0,1,2,3,4,5,6,7,8,9,...,952,953,954,955,956,957,958,959,960,961
0,1.000000,0.064767,0.089934,0.178503,0.083383,0.078191,0.163356,0.207574,0.148203,0.215520,...,0.020262,0.028345,0.042136,0.013539,0.041223,0.014169,0.028470,0.020262,0.028345,0.042136
1,0.064767,1.000000,0.196925,0.082430,0.290112,0.239536,0.119422,0.151578,0.093770,0.107711,...,0.009374,0.036990,0.020516,0.015729,0.058402,0.011138,0.044227,0.009374,0.036990,0.020516
2,0.089934,0.196925,1.000000,0.132827,0.370936,0.326090,0.144697,0.179885,0.164380,0.145057,...,0.019646,0.092944,0.074069,0.078994,0.078947,0.035575,0.092857,0.019646,0.092944,0.074069
3,0.178503,0.082430,0.132827,1.000000,0.134208,0.105407,0.179941,0.131130,0.121204,0.203588,...,0.032595,0.042822,0.096247,0.022550,0.048947,0.015444,0.068802,0.032595,0.042822,0.096247
4,0.083383,0.290112,0.370936,0.134208,1.000000,0.429790,0.156539,0.125673,0.095608,0.175908,...,0.019585,0.074562,0.053465,0.026779,0.092325,0.016468,0.094060,0.019585,0.074562,0.053465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,0.014169,0.011138,0.035575,0.015444,0.016468,0.029411,0.024141,0.017646,0.018658,0.016728,...,0.166377,0.038957,0.099748,0.071055,0.057504,1.000000,0.089690,0.166377,0.038957,0.099748
958,0.028470,0.044227,0.092857,0.068802,0.094060,0.081008,0.023128,0.035350,0.033543,0.036814,...,0.170887,0.112274,0.168196,0.149591,0.076318,0.089690,1.000000,0.170887,0.112274,0.168196
959,0.020262,0.009374,0.019646,0.032595,0.019585,0.017586,0.012697,0.013604,0.023353,0.031195,...,1.000000,0.048594,0.159558,0.089211,0.052980,0.166377,0.170887,1.000000,0.048594,0.159558
960,0.028345,0.036990,0.092944,0.042822,0.074562,0.133958,0.023009,0.045421,0.032169,0.035036,...,0.048594,1.000000,0.069243,0.119066,0.100679,0.038957,0.112274,0.048594,1.000000,0.069243


### Modelling

In [ ]:
df = pd.read_csv('clean_data1.csv')
df = df.drop(['Resume'],axis=1)
df.rename(columns={'newer_res':'Resume'},inplace=True)
resume_punc = df["Resume"].copy(deep  = True)
df.head()
#resume_punc

,Unnamed: 0,Category,Resume
0,0,Data Science,skills programming languages python pandas num...
1,1,Data Science,education details may 2013 may 2017 b e uit rg...
2,2,Data Science,areas interest deep learning control system de...
3,3,Data Science,skills r python sap hana tableau sap hana sql ...
4,4,Data Science,education details mca ymcaust faridabad haryan...


In [ ]:
import string
def rem_punc(s):
    punc = string.punctuation
    return [i for i in s if i not in punc]

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Remove punctaution for further processing
for ind,i in enumerate(df.itertuples()):
    token = nltk.word_tokenize(i[3])
    #print(token)
    df["Resume"][ind] = " ".join(rem_punc(token))

In [ ]:
import string
from wordcloud import STOPWORDS
def rem_punc(s):
    punc = string.punctuation
    return [i for i in s if i not in punc]

def rem_sw(s):
    sw = set(STOPWORDS)
    return [i for i in s if i not in sw]

def preprocess(eval_res):
    try:
        eval_res = eval(eval_res).decode()
    except:
        pass
    eval_res = eval_res.encode("ASCII","ignore").decode()
    length = len(eval_res)
    eval_res = " ".join(eval_res.split("\n"))
    token = rem_sw(nltk.word_tokenize(eval_res)) #Removing punctaution later since we need punctaution for sentence tokenization
    eval_res = " ".join(token).lower()
    return eval_res


### Cleaning data and adding in ID for category

In [ ]:
from io import StringIO
col = ['Category', 'Resume']
df = df[col]
df = df[pd.notnull(df['Resume'])]
df.columns = ['Category', 'Resume']
df['category_id'] = df['Category'].factorize()[0]
category_id_df = df[['Category', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Category']].values)

df.head()

,Category,Resume,category_id
0,Data Science,skills programming languages python pandas num...,0
1,Data Science,education details may 2013 may 2017 b e uit rg...,0
2,Data Science,areas interest deep learning control system de...,0
3,Data Science,skills r python sap hana tableau sap hana sql ...,0
4,Data Science,education details mca ymcaust faridabad haryan...,0


### Vectorizing docs¶

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1,2), stop_words='english')
features = tfidf.fit_transform(df.Resume).toarray()
labels = df.category_id
features.shape

(962, 27238)

### Applying Naive Bayes

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

x_train, x_test, y_train, y_test = train_test_split(df['Resume'], df['Category'], random_state = 0)

#print(x_train)

count_vect = CountVectorizer() # bag-of-ngrams model , based on frequency count
x_train_counts = count_vect.fit_transform(x_train)

tfidf_transformer = TfidfTransformer() #passing the word:word count
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

classifier = MultinomialNB().fit(x_train_tfidf, y_train)

### Checking Accuracy of Naive Bayes Model


In [ ]:
df = pd.read_csv('clean_data1.csv')
df = df.drop(['Resume'],axis=1)
df.rename(columns={'newer_res':'Resume'},inplace=True)
df.head()

,Unnamed: 0,Category,Resume
0,0,Data Science,skills programming languages python pandas num...
1,1,Data Science,education details may 2013 may 2017 b e uit rg...
2,2,Data Science,areas interest deep learning control system de...
3,3,Data Science,skills r python sap hana tableau sap hana sql ...
4,4,Data Science,education details mca ymcaust faridabad haryan...


In [ ]:
from io import StringIO
col = ['Category', 'Resume']
df = df[col]
df = df[pd.notnull(df['Resume'])]
df.columns = ['Category', 'Resume']
df['category_id'] = df['Category'].factorize()[0]
category_id_df = df[['Category', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Category']].values)

In [ ]:
res_train, res_test, cat_train, cat_test = train_test_split(df['Resume'], df['Category'], test_size=0.3, random_state = 15)

vectorizer= CountVectorizer()
res_counts= vectorizer.fit_transform(res_train)

tfidf= TfidfTransformer()
res_tfidf= tfidf.fit_transform(res_counts)


classifier=MultinomialNB().fit(res_tfidf, cat_train)

In [ ]:
predicted = []
for i in res_test:
    predicted.append((classifier.predict(vectorizer.transform([i])))[0])

In [ ]:
a= pd.DataFrame(cat_test)
a['predicted'] = predicted
a.head()

,Category,predicted
42,HR,HR
481,Electrical Engineering,Electrical Engineering
91,Advocate,Advocate
486,Electrical Engineering,Electrical Engineering
936,Testing,Testing


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
accuracy=accuracy_score(a.Category, a.predicted)
print("Accuracy from Naive bayes:",accuracy)

Accuracy from Naive bayes: 0.9238754325259516
